In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import StatevectorSampler,StatevectorEstimator
from qiskit.circuit.library import UnitaryGate
from qiskit.visualization import plot_histogram
from qiskit.compiler import transpile
import qiskit.quantum_info as qi
from qiskit.quantum_info.operators import SparsePauliOp
from itertools import product
from scipy.linalg import expm
import numpy as np
import matplotlib.pyplot as plt

**WARNING**

The IBM platform to monitor jobs on real devices is migrating. We will use the old version for this project, available at this link : https://quantum.ibm.com

## The HHL algorithm 

We want to implement the quantum circuit that allows us to solve linear systems of equations. We want to solve $Ax=b$ with full rank the matrix $A \in \mathbb{C}^{ 2^n\times 2^n}$ and a vector $b \in \mathbb{C}^{ 2^n}$.

For that, we encode $b$ in a state $|b \rangle$ such that $|b\rangle = \sum_{i=0}^{2^n-1} b_i |i\rangle$. Here, the $b_i$ are the components of $b$ and $|i\rangle$ is the state of the computational basis given by the binary decomposition of $i$. 

The solution x is then given by $ |x\rangle = cA^{-1}|b\rangle, \quad c^{-1} =\|A^{-1}|b\rangle\|$. 

We have shown in the handout that if $(|u_i\rangle)_{i=0, \dots,2^n-1}$ is an eigenbasis of $A$ associated to the eigenvalues $\lambda_i$ then we can rewrite $ |b\rangle = \sum_{i=0}^{2^n-1} \beta_i |u_i\rangle $ and the state $|x\rangle$ is $|x\rangle = c \sum_{i=0}^{2^n-1} \frac{\beta_i}{\lambda_i} |u_i\rangle$.

The circuit we want to implement is the following: 

<img src="./circuit.png" alt="drawing" width="800">

To have an easy implementation of the algorithm, we impose several constraints on $A$: 

- A is Hermitian.
- The eigenvalues of $A$ are powers of 2. 

We give below a function that allows you to sample a matrix $A$ that verifies those constraints. 

In [ ]:
def sample_A(n,i):
    """
    Sample a random n-qubit Hermitian matrix A with eigenvalues being powers of 2 and at most 2^{i-1}.
    """
    G = np.triu(np.random.rand(2**n, 2**n)) + 1j * np.triu(np.random.rand(2**n, 2**n))
    H = G + G.T.conj()
    np.fill_diagonal(H, np.random.rand(2**n))
    _, v = np.linalg.eigh(H)
    return v @ np.diag(np.random.choice([2**j for j in range(i)], 2**n)) @ v.T.conj()  # random

### The QPE 

Let's start with the quantum phase estimation part. 

The detailed circuit is the following: 

<img src="./qpe.png" alt="drawing" width="600">


To implement the inverse QPE, we will need the inverse gates. Thus, we will construct both every time.


#### Hadamard Layer 

We start by constructing a layer of Hadamard gates. Complete the following cell. 

In [ ]:
def Hadamard_layer(m):
    """
    Returns a quantum circuit that applies the Hadamard gate to each of the m qubits in the circuit.
    """
    qc = QuantumCircuit(m, name="$H^{\otimes m}$")
    ...
    return qc

In [ ]:
qc = Hadamard_layer(3)
qc.draw('mpl')

Let's check that our function is correct by applying it to the state |0⟩⊗3 and checking that the resulting state is the uniform superposition over all 3-qubit strings.

To draw the output state, `qiskit` can require the package `pylatexenc`. To install it, you can use `pip install pylatexenc`.

In [ ]:
# run the circuit with initial state |0>^{\otimes 3} 
output = qi.Statevector.from_instruction(qc)
uniform = np.ones(2**3)/np.sqrt(2**3)

display(output.draw('latex'))

print("The output state is uniform :", output.equiv(uniform))

The Hadamard gate is its own inverse, since $HH^\dagger = H^2 = I$. 

#### Controlled Unitary

We need now to implement the controlled version of the gate $U^{2^i} $ for $0 \leq i < m$. Complete the following cell. 

Hint : Let's recall that $U = e^{i 2\pi\frac{A}{2^m}}$. The exponential of a matrix can be computed using `expm` from the `sympy` library. Arbitrary unitary can be added to the circuit with `UnitaryGate` from the qiskit library. 

In [ ]:
def controlled_Us(A, n, m):
    """
    Returns a quantum circuit that applies the m controlled-U^{2^i} gates 
    U is a 2^n x 2^n matrix, and m is the number of control qubits.
    U is U = e^{i 2 pi A/2^m} where A is a Hermitian matrix.
    """
    qc = QuantumCircuit(n + m, name="cU")
    ...
    return qc

We also need the inverse operation. For that, we need to add the **inverse** of the controlled $U^{2^i} $ in **reverse** order. 

Think about what is the inverse of $U$.

In [ ]:
def inverse_controlled_Us(A, n, m):
    """
    Returns a quantum circuit that applies the m controlled-U^{2^i} gates 
    U is a 2^n x 2^n matrix, and m is the number of control qubits.
    U is U = e^{i 2 pi A/2^m} where A is a Hermitian matrix.
    """
    qc = QuantumCircuit(n + m, name="c$U^\dagger$")
    ...
    return qc

Let's draw the circuit.

In [ ]:
# Example usage
n = 2 #nb of qubits for A 
m = 4 #nb of memory qubits
A = sample_A(n, m)
qc = controlled_Us(A, n, m)
display(qc.draw('mpl'))

And check that applying our two functions one after the other gives the identity.

In [ ]:
qc = QuantumCircuit(m+n)

qc.append(controlled_Us(A, n, m), range(m+n))
qc.append(inverse_controlled_Us(A,n,m), range(m+n))
display(qc.decompose().draw('mpl'))

# Convert the circuit to a unitary matrix
U_circuit = qi.Operator(qc)

print("The circuit is equivalent to the Identity:" , np.allclose(U_circuit.data, np.eye(2**(m+n))))


#### QFT

To finish the QPE, we need to implement the QFT. Do not use `qiskit QFT` other than to check your result (the qubit ordering is different). 

In [ ]:
def QFT(m):
    """
    Returns a quantum circuit implementing the Quantum Fourier Transform on m qubits
    """
    qc = QuantumCircuit(m, name=r"$QFT$")
    ...
    return qc

And the inverse QFT.

In [ ]:
def inv_QFT(m):
    """
    Returns a quantum circuit implementing the inverse Quantum Fourier Transform on m qubits
    """
    qc = QuantumCircuit(m, name=r"$QFT^\dagger$")
    ...
    return qc

Let's draw the circuit.

In [ ]:
# Example usage
m = 3
qc = QFT(m)
qc.draw(output="mpl")

Let's check that $QFT|k\rangle =\frac{1}{\sqrt{2^m}} \sum_{j=0}^{2^m-1} e^{i2\pi \frac{kj}{2^m} }|j\rangle$. You can also check yourself that $QFT\, QFT^\dagger= I $

In [ ]:
# Choose any state in the computational basis. We chose |k> = |5 > for the example.
k = 5
state = qi.Statevector.from_int(k,2**m)
qc = QuantumCircuit(m)
qc.initialize(state, range(m))
qc.append(QFT(m), range(m))
output = qi.Statevector.from_instruction(qc)

expected = np.exp(2j * np.pi * k *  np.arange(2**m) /2**m) / np.sqrt(2**m)

print("The output state is the QFT of the input state:", output.equiv(expected))

Let's construct our QPE circuit at last. We do not add the ancilla qubit for now. 

In [ ]:
def QPE(A, n, m):
    """
    Returns a quantum circuit that implements the Quantum Phase Estimation algorithm
    A is a 2^n x 2^n matrix, and m is the number of memory qubits.
    """
    qc = QuantumCircuit(n + m)
    ...
    return qc

We will now test the QPE algorithm on a simple example. Let's consider the matrix $A = diag(1,2,4,8)$. The eigenvalues are $2^0,2^1,2^2,2^3$ associated to the eigenvectors $|00\rangle = |0\rangle, |01\rangle=|1\rangle,|10\rangle = |2\rangle,|11\rangle = |3\rangle$.
If we choose $|b\rangle = |i\rangle$ one of the eigenvectors, then we expect the output to be $|2^i\rangle \otimes |i\rangle $.

In the next cell, we choose $|b\rangle = |1 \rangle$, but you can change it to the one you want.


In [ ]:
n = 2 #nb of qubits for A 
m = 4 #nb of memory qubits
A = np.diag([1,2,4,8])

qpe_circuit = QuantumCircuit(n + m)
# Initialize the state |b> in the computational basis; here we choose |b> = |1> = | 01>
qpe_circuit.prepare_state("01", range(m,n+m))
qpe_circuit = qpe_circuit.compose(QPE(A, n, m).decompose())
qpe_circuit.draw(output="mpl",fold=-1)


**Warning**: Since qiskit changes the order of the qubits, we should instead observe $|i\rangle \otimes |2^i\rangle $. 

With our example, we should observe $|1\rangle \otimes |2\rangle = |01\rangle \otimes |0010\rangle $

In [ ]:
output = qi.Statevector.from_instruction(qpe_circuit)
display(output.draw('latex'))

### The controlled rotation 

The central part of the algorithm is the control rotation that realizes the transformation $ R(|0\rangle \otimes |\lambda\rangle) = \left(\sqrt{1-\frac{1}{\lambda^2}}|0\rangle + \frac{1}{\lambda}|1\rangle\right) \otimes |\lambda\rangle $.

#### But how to implement this operation ?

First, note that since the eigenvalues are powers of 2, the only eigenvalue that has a $1$ at the $i$-th position in its binary decomposition is $2^i$. We also assume that $\max_i (\lambda_i) \leq 2^{m-1}$. Thus, we can use the qubit $i$ to know if the memory register is in the state $|2^i\rangle $ or not. We will use $m$ controlled $R_y(\theta_i)$ gates, one per possible eigenvalue, to implement $R$. 

Remember that the $R_y$ gate is the unitary given by $R_y(\theta) = \left( \begin{array}{cc} cos(\theta/2) & -sin(\theta/2)\\ sin(\theta/2) & cos(\theta/2) \end{array}\right)$. 

Each controlled rotation will act on the first ancilla qubit and be controlled by the $i$-th qubit of the memory register, we denote them $c_iR_y(\theta_i)$.

Thus, we have the decomposition $ R = c_1R_y(\theta_1) \dots c_m R_y(\theta_m)$.

What value of $\theta_i$ should you choose to obtain $ R(|0\rangle \otimes |2^i \rangle) = \left(\sqrt{1-\frac{1}{2^{2i}}}|0\rangle + \frac{1}{2^i}|1\rangle\right) \otimes |2^i\rangle $ ? 



In the next cell, we choose i = 2 for the test. 

**Warning**: **Be careful with the ordering of the qubits!**

In our handout, the ancilla register comes first, then comes the memory register: $ |0\rangle |2^2 \rangle $ 

In qiskit, the order is reversed: $ |2^2 \rangle |0\rangle$.

In our handout, the binary decomposition is written from left to right, from the most significant bit to the least significant bit : $|4\rangle = |100\rangle$.

In qiskit, the order is reversed. 

So the input state is $|0\rangle_{q_0} |2^2 \rangle_{q_1q_2q_3}  = |1 \rangle_{q_3}|0 \rangle_{q_2}|0 \rangle_{q_1} |0\rangle_{q_0}$

In [ ]:
def R(m):
    """
    Returns a quantum circuit that implements the R operator
    R = \prod_{i=1}^m C_i R_y
    where R_y^i is a controlled rotation gate.
    """
    qc = QuantumCircuit(m + 1, name="R")
    ...
    return qc

In [ ]:
qc_R = QuantumCircuit(4)
qc_R.initialize("100",[1,2,3])  # input state |100> = |4> = |2^2>
qc_R.barrier()
qc_R = qc_R.compose(R(3), range(4))
display(qc_R.draw(output="mpl"))

 For the output state, with qiskit ordering, we should observe $\sqrt{1 - 2^{-2i}}|2^i\rangle |0\rangle + 1/ 2^i  |2^i\rangle |1\rangle$. 

In our case, we should observe $\sqrt{15/16}|2^2\rangle |0\rangle + 1/ 2^2  |2^2 \rangle |1\rangle = \sqrt{15/16}|100\rangle |0\rangle + 1/ 4  |100 \rangle |1\rangle$.

In [ ]:
output = qi.Statevector.from_instruction(qc_R)
display(output.draw('latex'))

### Complete circuit

We are ready to implement the complete circuit.

Complete the next cell with the HHL circuit **without** the last measurement. 

In [ ]:
def hhl_circuit(A, b, n, m):
    """
    Returns a quantum circuit that implements the HHL algorithm
    A is a 2^n x 2^n Hermitian matrix, and b is a vector of size 2^n.
    The circuit uses 1 ancilla qubits and m memory qubit and n qubits for b.
    """
   
    qc = QuantumCircuit(1+ n + m, name="Complete circuit")
    ...
    return qc

Let's test it with a random $A$. If you need to debug your code, you might prefer reusing the previous diagonal $A$. 

In [ ]:
n = 2
m = 2
A = sample_A(n,m)
b = np.array([1,0, 1, 1j]) 
b = b / np.linalg.norm(b)


final_circuit = hhl_circuit(A, b,n, m)
display(final_circuit.draw(output="mpl",fold=-1))
display(final_circuit.decompose().draw(output="mpl", fold=-1))

#### Final state

We should observe that the memory qubits are all 0 as predicted.

In [ ]:
output = qi.Statevector.from_instruction(final_circuit)
display(output.draw('latex'))

The final state of the circuit is $ |0 \rangle |0^{\otimes m} \rangle |y\rangle + |1\rangle |0^{\otimes m} \rangle|x\rangle $ where $|x\rangle $ is the solution of the system and $|y\rangle$ is not. 

Let extract $|x \rangle$ from the output state to verify that it's a solution to our system of equations. 

**Warning**: since the order of the qubits is reversed, what you really observed in the output state is $|y\rangle|0^{\otimes m} \rangle |0 \rangle+ |x\rangle |0^{\otimes m} \rangle |1\rangle $!

*Hint*: $$|a\rangle|0\rangle|0\rangle =  \left( \begin{array}{c} a_1 \\ a_2 \end{array}\right) \otimes \left( \begin{array}{c} 1 \\ 0\\ 0\\0  \end{array}\right)  = \left( \begin{array}{c} a_1 \\ 0\\ 0\\0 \\ a_2 \\ 0\\ 0\\0  \end{array}\right) $$

In [ ]:
output_state = output.data
x = qi.Statevector(...)

expected_solution = np.linalg.solve(A, b)
expected_solution = expected_solution / np.linalg.norm(expected_solution)

print("The output state is the expected solution:", x.equiv(expected_solution))


If we measure the ancilla qubit, what is the probability to obtain "1"?

In [ ]:
p = ...

print("The probability of success is: ", p)

Let's simulate the circuit to verify that we obtain the same value.
Add the measurement on the ancilla qubit to your circuit.

In [ ]:
circuit_with_measurements = QuantumCircuit( n + m + 1, 1)
...

Set up your simulator using a `StatevectorSampler`.

**Warning**: When you measure all qubits, the result of the measure is in the attribute `data.meas` of the job result. When you measure only a few qubits, the result of the measure is in the attribute `data.c`.

In [ ]:


counts = ...

display(plot_histogram(counts))
print("The counts:" , counts)
print("The probabilities:" , ...)


### Reconstructing $|x\rangle$ 



For this part, we will solve the specific system given in the handout. The eigenvalues of $A$ are only $1$ and $2$.

Start by creating the circuit with the appropriate $A$ and $b$.  Add the measurements for the ancilla qubit and the $|x\rangle$ register. 

Compute also the expected solution and the success probability.

In [ ]:
n = 1
m = 2

A = ...
b = ...

final_circuit =  ...

final_circuit_with_measurements = ...

expected_solution = ...

p = ... 

print("The probability of success is: ",  p)
print("The expected solution is:", expected_solution)
display(final_circuit.decompose().draw(output="mpl",fold=-1))

Use a `StatevectorSampler` to measure the first and last qubit. 

In [ ]:

counts = ...

display(plot_histogram(counts))
print("The counts:" , counts)
print("The probabilities:" ,  ...)

Try reconstructing $|x\rangle$ from the output of the measurement. What is missing to completely determine $|x\rangle$?

In [ ]:
x_simulator = ...

Now let see what happens on a real device. 

First, make sure `qiskit_ibm_runtime` is installed. If not, run `pip install qiskit-ibm-runtime` in your Python environment. Then, set up to use IBM Quantum Platform according to the [guide](https://docs.quantum.ibm.com/guides/setup-channel#set-up-to-use-ibm-quantum-platform).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

Let's choose a backend. Below, we are using the least busy one, but you can pre-set it if you want.

In [ ]:
backend = service.least_busy(operational=True)
print(backend)

Now, we want to send the circuits to a QPU. However, before this, we have to adapt them to the architecture of the chosen computer, as it might have a specific topology and a limited number of implemented basis gates. Transpile your circuit to adapt it to the backend architecture. 


In [ ]:
backend.configuration().basis_gates

In [ ]:
transpiled_qc  = ...
print(f'Depth of circuit after transpiling: {transpiled_qc.depth()}')
transpiled_qc.draw(output="mpl",idle_wires=False)

Now, we instantiate the [`Sampler`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.SamplerV2) interface which allows us to use the backend for measurement.

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = ...

n_shots = ...

job = ...
print(f">>> Job ID: {job.job_id()}")

In [ ]:
counts = ...

print("The counts:" , counts)
print("The probabilities:" , {key: value/n_shots for key, value in counts.items()})

Reconstruct $|x\rangle$. Compare the result to the one obtained with the simulator. 

In [ ]:
x_real = ...

print("Error between the simulated solution and the real device one", np.linalg.norm(x_real - x_simulator))

### Bonus

With a real quantum device, we don't have direct access to the state. We can only gain information through measurement. 


Start by creating the circuit with the new $A$ and $b$ without the ancilla measurement. Compute also the expected solution and the probability of success. 

In [ ]:
n = 1
m = 2

A = ...
b = ...


final_circuit = ...

p = ...

print("The probability of success is: ", p)

expected_solution = ...


print("The expected solution is:", expected_solution)
display(final_circuit.decompose().draw(output="mpl",fold=-1))

Let's create the 4 observables $\sigma \in \{I,X,Y,Z\}$ using `SparsePauliOp`. 

In [ ]:

X = ...
Z = ...
Y = ...
I = ...

To do the measure on the real device, we need to define the observables for the full circuit, so for the 4 qubits. 

We want to measure the observables only when the result of the measurement on the ancilla qubit is 1.  Thus, the observable for the ancilla qubit will be $P_1 = |1\rangle \langle 1|$. We are not interested in the 2 memory qubits since they are in the state $|0\rangle^{\otimes 2}$, so the final observables will be $P_1 \otimes I \otimes I \otimes \sigma$ instead of just $\sigma$. 
Adapt your previous operators and store them in the list `observables`. You can use the `expand()` method of the operators.


In [ ]:
P1 = ...

observables = ...

#### Simulator 

We start by running our circuit on the simulator.

Set up your simulator using a [`StatevectorEstimator`](https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.StatevectorEstimator) since this time we are measuring observables. 

The result of the measure is in the attribute `data.evs` of the job result (for expectation values). It's a list containing the expectation values $\langle P_1 \otimes I_2 \otimes I_2 \otimes \sigma\rangle$ for $\sigma \in \{I,X,Y,Z\}$.


In [ ]:
estimator_perfect = StatevectorEstimator(default_precision=1e-10)

data_simulated = ...
print("The estimated expectation values are: ", data_simulated)

Reconstruct the matrix $\rho$ from the results of the measurement: 
    $$\rho = \frac{\langle I \rangle I + \langle X \rangle X+\langle Y \rangle Y+\langle Z \rangle Z}{\sqrt{2}}$$

Hint : $Tr (A \otimes B)$ = $Tr(A) Tr(B)$ and verify that $Tr(\rho) =1$

In [ ]:
rho = ...

Since $\rho =|x\rangle \langle x|$, compute $|x\rangle$ and check the error with the expected solution. Don't forget about the global phase.
Compare the obtained error with the `default_precision` parameter of the `StatevectorEstimator`. 

In [ ]:
x_reconstructed = ...

print("Error between the simulated solution and the expected one", np.linalg.norm(x_reconstructed - expected_solution))

#### Real device

Now we are going to make the observable measurements on a real quantum computer.	



Let's choose a backend. Below, we are using the least busy one, but you can pre-set it if you want.


In [ ]:
backend = service.least_busy(operational=True)
print(backend)

Now, we want to send the circuits to a QPU and measure the observables. However, before this, we have to adapt them to the architecture of the chosen computer, as it might have a specific topology and a limited number of implemented basis gates. Transpile your circuit to adapt it to the backend architecture. 


In [ ]:
backend.configuration().basis_gates

In [ ]:
transpiled_qc = ...
print(f'Depth of circuit after transpiling: {transpiled_qc.depth()}')
transpiled_qc.draw(output="mpl",idle_wires=False)

The real device has more qubits than what we need. So we also have to adapt the observables to the updated circuit. The next cell shows how it can be done.

Remark that it consists of adding $I$ on all the qubit we don't use.

In [ ]:
observables_layout = [ O.apply_layout(transpiled_qc.layout) for O in observables]
print("The layout of the observables is:", observables_layout[0])

Now, we instantiate the [`Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) interface which allows us to use the backend for observables measurement.

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator_hardware = Estimator(mode=backend)

Finally, we can submit the circuit, which is done in the same fashion as in the tutorial. Fill in the argument with the transpiled circuits and corresponding observables.

The real circuit, unlike the perfect simulator, estimates $\langle \sigma \rangle$ through actual measurements rather than the mathematical operations. The number of shots is indirectly specified through the `precision` argument &mdash; it is of order $O(\frac{1}{\sqrt{\mathrm{n\_shots}}})$. The lower is its value, the higher is the number of shots used.  Default is usually set to $0.015625 = \frac{1}{\sqrt{4096}}$, i.e. 4096 shots ([source](https://github.com/Qiskit/qiskit-ibm-runtime/blob/639f3071833cef46443adf83eba7ea925b00257c/qiskit_ibm_runtime/options/estimator_options.py#L39)).
 

In [ ]:
job = estimator_hardware.run([(transpiled_qc, observables_layout)])

# Use the job ID to retrieve your job data later
print(f">>> Job ID: {job.job_id()}")

Compare the results with the simulated data. To retrieve the job results, do like in the simulation.

In [ ]:
data_estimated = ...
print("The estimated expectation values are: ", data_estimated)
print("The simulated expectation values are: ", data_simulated)

Reconstruct the matrix $\rho$ and the state $|x\rangle$. 

How close are you from the real solution? Comment your result.

In [ ]:
rho_hardware = ...
x_hardware = ...


print("Error between the noisy solution and the expected one", np.linalg.norm(x_hardware - expected_solution))